In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredEPubLoader
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


import pinecone
from pinecone import ServerlessSpec
import time

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY") #langchain searches for this key and loads it automatically
os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY") #langchain searches for this key and loads it automatically

In [2]:
# Load PDF File
loader = PyPDFLoader('./assets/srp-covid-19-6month.pdf')
pdf = loader.load()

In [ ]:
# Pinecone index name
index_name = "rag-project"

# embeddings using OpenAI "text-embedding-ada-002"
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Create the vector store
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name, 
    embedding=embed_model, 
    text_key="text", # the key containing the text chunks in the meta data
    namespace="all-users", # optional if you didn't set a name space
    pool_threads=4
)

In [16]:
# Creat the QA Chain
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever()
)

query = "When was thef first covid case?"

qa.invoke(query)



{'query': 'When was thef first covid case?',
 'result': 'The first known cases of COVID-19 were reported in December 2019 in Wuhan, China.'}

#### Recievers

In [ ]:
# Import Ollama LLM
from langchain_ollama import OllamaLLM

# Load Ollama LAMA2 LLM Model
llm = OllamaLLM(model='llama2 ')

# Design Chat Prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Answer the following question based only on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context>
        {context}
    </context>
    Question: {input}""")

# Chain Introduction

# Create Stuff Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
 
document_chain = create_stuff_documents_chain(llm, prompt)

# Retrievers
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain.invoke({"input": query})